In [16]:
library(tidyverse)
library(reshape2)
library(matrixStats)
library(dplyr)
library(Seurat)
library(gplots)
library(ggplot2)
library(purrr)
library(DropletUtils)
library(SingleCellExperiment)
library(scater)
library(Matrix)
library(celldex)
library(gridExtra)
library(mixtools)
library(enrichR)
library(stringr)
library(hash)
library(enrichR)
library(ggplot2)
library(dplyr)
library(stringr)
# Load necessary libraries
library(ggplot2)
library(viridis)

In [84]:
df = read.csv("ranked_interactions.txt", sep = "\t")

df = subset(df,fdr_ligand <= 0.05)
df = subset(df,numSigI1_fdr05_receptor >= 5)
df = subset(df, log2FC_ligand >=0.01 | log2FC_ligand <= -0.01)
df$interaction <- apply(df[, c("cell_type_ligand", "cell_type_receptor")], 1, function(x) {
  paste(sort(x), collapse = ";")
})


pdf("cGASKO-PD1_GSEA.pdf", width = 12, height = 6)


for (x in unique(df$interaction)){
    print(x)
    
    interaction_df = subset(df,interaction == x)
    pos_df <- subset(interaction_df, log2FC_ligand > 0 )
    pos_genes <- unique(c(pos_df$ligand, pos_df$receptor))

    
    # Load Enrichr
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes
    websiteLive <- TRUE
    dbs <- listEnrichrDbs()
    if (is.null(dbs)) websiteLive <- FALSE
    if (websiteLive) head(dbs)
    # read signature
    dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
    term_list = c()
    if (websiteLive & length(pos_genes) > 2) {
      enriched <- enrichr(pos_genes, dbs)
    
        mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
        dim(mut_enr_ch)
        dim(mut_enr_go)
        mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)
        dim(mut_enr)
        write.csv(mut_enr,file="mut_enr_metagenes.csv")
        mp = "Meta-program n"
        cohort = "Sarcoma"
        enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
        topn = "Top100"
        pos_mut_enr <- mut_enr
        mut_enr_p <- subset(mut_enr, Adjusted.P.value < 0.05)
        
        h_mut_enr1 <- mut_enr_p[order(mut_enr_p$qscore, decreasing = TRUE), ][1:10,]#[1:500,]

        pos_gene_sets <- h_mut_enr1
        term_list <- c(term_list,pos_gene_sets$Term)
    }
    
    
    neg_df <- subset(interaction_df, log2FC_ligand < 0 )
    neg_genes <- unique(c(neg_df$ligand, pos_df$receptor))
    print(neg_genes)
    
    
      # Load Enrichr
    listEnrichrSites()
    setEnrichrSite("Enrichr") # Human genes
    websiteLive <- TRUE
    dbs <- listEnrichrDbs()
    if (is.null(dbs)) websiteLive <- FALSE
    if (websiteLive) head(dbs)
    # read signature
    dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
    
    if (websiteLive & length(neg_genes) > 2) {      
        enriched <- enrichr(neg_genes, dbs)
    
        mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
        mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
        dim(mut_enr_ch)
        dim(mut_enr_go)
        mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)
        dim(mut_enr)
        write.csv(mut_enr,file="mut_enr_metagenes.csv")
        mp = "Meta-program n"
        cohort = "Sarcoma"
        enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
        topn = "Top100"
        neg_mut_enr <- mut_enr
        mut_enr_p <- subset(mut_enr, Adjusted.P.value < 0.05)

        h_mut_enr1 <- mut_enr_p[order(mut_enr_p$qscore, decreasing = TRUE), ][1:10,]#[1:500,]
        neg_genes_sets <- h_mut_enr1
        term_list <- c(term_list,rev(neg_genes_sets$Term))
    }
    print(term_list)
    term_list <- term_list[!duplicated(term_list)]
    neg_subset = subset(neg_mut_enr, Term %in% term_list)
    pos_subset = subset(pos_mut_enr, Term %in% term_list)

    # Ensure the term_list has no duplicates
    term_list <- term_list[!duplicated(term_list)]

    # Create the pos_subset and neg_subset dataframes
    neg_subset <- subset(neg_mut_enr, Term %in% term_list)
    pos_subset <- subset(pos_mut_enr, Term %in% term_list)

    # Merge the two dataframes on the Term column
    combined_df <- merge(pos_subset[, c("Term", "qscore", "Adjusted.P.value")], 
                         neg_subset[, c("Term", "qscore", "Adjusted.P.value")], 
                         by = "Term", 
                         all = TRUE, 
                         suffixes = c("_pos", "_neg"))
    # Replace NA values with 0
    combined_df[is.na(combined_df)] <- 0

    # Set Term as a factor with levels in the order of term_list
    combined_df$Term <- factor(combined_df$Term, levels = term_list)

    # Order the dataframe by the Term factor
    combined_df <- combined_df[order(combined_df$Term), ]
    combined_df <- combined_df[rev(rownames(combined_df)), ]


    # Print the combined dataframe
    print(combined_df)
    
    data_long <- data.frame(
      Term = rep(combined_df$Term, 2),
      qscore = c(combined_df$qscore_pos, -combined_df$qscore_neg),
      pvalue = c(combined_df$Adjusted.P.value_pos, combined_df$Adjusted.P.value_neg),
      group = rep(c("Positive", "Negative"), each = nrow(combined_df))
    )

    # Adjust the p-values: take -log(pvalue) and make negative values for the "Negative" group
    data_long$pvalue <- -log10(data_long$pvalue)
    data_long$pvalue[is.na(data_long$pvalue) | is.infinite(data_long$pvalue)] <- 0

    data_long$pvalue[data_long$group == "Negative"] <- -data_long$pvalue[data_long$group == "Negative"]

    # Plot the data
    data_long$color_group <- with(data_long, ifelse(group == "Positive", "red", "blue"))
    max_pvalue <- max(abs(data_long$pvalue))


    # Plot the data
    p<- ggplot(data_long, aes(fill = pvalue, y = qscore, x = Term)) + 
      geom_bar(position = "stack", stat = "identity") + 
      scale_fill_gradientn(colors = c("darkblue", "lightblue", "lightpink", "darkred"),
                           values = scales::rescale(c(-max_pvalue, 0, max_pvalue)),
                           name = "-log10(P-Value)", limits = c(-max_pvalue, max_pvalue)) +
      geom_hline(yintercept = 0, color = "black", size = 1) + 

      ggtitle(paste(x, " interactions, GSEA cGASKO-PD1"))+
      ylab("Q Score") + 
      xlab("Term") +
      coord_flip() +  # Flip coordinates if you want horizontal bars
      theme_minimal()

    print(p)

}
dev.off()

[1] "Malignant;Malignant"


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
 [1] "Sema6a" "Sema6d" "Ereg"   "Tnc"    "Areg"   "Nrg1"   "Tgfa"   "Pdgfd" 
 [9] "Fgf7"   "Sema3e" "Col8a1" "Apoe"   "Cp"     "Ngf"    "Igfbp5" "Slit3" 
[17] "Slit2"  "Il1b"   "Il1a"   "Il1rn"  "Vwf"    "Fgf9"   "Fgf13"  "Gas6"  
[25] "Lgals3" "Plxna4" "Ccbe1"  "Egfr"   "Pdgfrb" "Nrp1"   "Gpc4"   "Sorcs2"
[33] "Hmmr"   "Sdc2"   "Itga2"  "Plxna2" "Insr"   "Fgfr1"  "Mertk" 


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
 [1] "transmembrane receptor protein tyrosine kinase signaling pathway (GO:0007169)"
 [2] "regulation of cell migration (GO:0030334)"                                    
 [3] "positive regulation of cell migration (GO:0030335)"                           
 [4] "positive regulation of cell motility (GO:2000147)"                            
 [5] "positive regulation of phosphorylation (GO:0042327)"                          
 [6] "regulation of phosphatidylinositol 3-kinase signaling (GO:0014066)"           
 [7] "semaphorin-plexin signaling pathway (GO:0071526)"                             
 [8] "positive regulation of phosphatidylinositol 3-kinase signaling (GO:0014068)"  
 [9] "positive regulation of protein phosphorylation (GO:0001934)"                  
[10] "Epithelial Mesenchymal Transition"                                            
[1

...

Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
[1] "Nid2"  "Hspg2" "Nrp1" 


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



Uploading data to Enrichr... Done.
  Querying MSigDB_Hallmark_2020... Done.
  Querying GO_Biological_Process_2021... Done.
Parsing results... Done.
 [1] "regulation of axon extension involved in axon guidance (GO:0048841)"                    
 [2] "semaphorin-plexin signaling pathway (GO:0071526)"                                       
 [3] "neural crest cell migration (GO:0001755)"                                               
 [4] "dendrite arborization (GO:0140059)"                                                     
 [5] "neuron projection extension involved in neuron projection guidance (GO:1902284)"        
 [6] "axon extension involved in axon guidance (GO:0048846)"                                  
 [7] "sympathetic ganglion development (GO:0061549)"                                          
 [8] "ganglion development (GO:0061548)"                                                      
 [9] "axon guidance (GO:0007411)"                                                           

png 
  2

In [80]:


# Assuming your dataframe is named combined_df and contains columns:
# Term, qscore_pos, qscore_neg, Adjusted.P.value_pos, Adjusted.P.value_neg

# Reshape data for plotting


In [30]:
# Ensure the term_list has no duplicates
term_list <- term_list[!duplicated(term_list)]

# Create the pos_subset and neg_subset dataframes
neg_subset <- subset(neg_mut_enr, Term %in% term_list)
pos_subset <- subset(pos_mut_enr, Term %in% term_list)

# Merge the two dataframes on the Term column
combined_df <- merge(pos_subset[, c("Term", "Combined.Score", "Adjusted.P.value")], 
                     neg_subset[, c("Term", "Combined.Score", "Adjusted.P.value")], 
                     by = "Term", 
                     all = TRUE, 
                     suffixes = c("_pos", "_neg"))

# Rename columns for clarity
names(combined_df)[names(combined_df) == "combined_score_pos"] <- "pos_combined_score"
names(combined_df)[names(combined_df) == "combined_score_neg"] <- "neg_combined_score"
names(combined_df)[names(combined_df) == "pvalue_pos"] <- "pos_pvalue"
names(combined_df)[names(combined_df) == "pvalue_neg"] <- "neg_pvalue"

# Replace NA values with 0
combined_df[is.na(combined_df)] <- 0

# Set Term as a factor with levels in the order of term_list
combined_df$Term <- factor(combined_df$Term, levels = term_list)

# Order the dataframe by the Term factor
combined_df <- combined_df[order(combined_df$Term), ]

# Print the combined dataframe
print(combined_df)

# Print the combined dataframe
print(combined_df)


                                                            Term
1                                      axonogenesis (GO:0007409)
2       cellular response to growth factor stimulus (GO:0071363)
3                central nervous system development (GO:0007417)
4                                           IL-2/STAT5 Signaling
5                             inflammatory response (GO:0006954)
6                                              KRAS Signaling Up
7  plasma membrane bounded cell projection assembly (GO:0120031)
8             positive regulation of cell migration (GO:0030335)
9      positive regulation of ERK1 and ERK2 cascade (GO:0070374)
10              positive regulation of MAPK cascade (GO:0043410)
11           positive regulation of phosphorylation (GO:0042327)
12       positive regulation of signal transduction (GO:0009967)
13                       regulation of angiogenesis (GO:0045765)
14              regulation of ERK1 and ERK2 cascade (GO:0070372)
15      regulation of neu

In [13]:
library(enrichR)
library(ggplot2)
library(dplyr)
library(stringr)
# Load Enrichr
listEnrichrSites()
setEnrichrSite("Enrichr") # Human genes
websiteLive <- TRUE
dbs <- listEnrichrDbs()
if (is.null(dbs)) websiteLive <- FALSE
if (websiteLive) head(dbs)
# read signature
mylist <-  read.csv(file="t100_genes_per_metaprogram.csv")
dbs <- c("MSigDB_Hallmark_2020","GO_Biological_Process_2021")
if (websiteLive) {
  enriched <- enrichr(genes, dbs)
}
mut_enr_ch<-mutate(enriched[[1]], qscore = -log(Adjusted.P.value, base=10))
mut_enr_go<-mutate(enriched[[2]], qscore = -log(Adjusted.P.value, base=10))
dim(mut_enr_ch)
dim(mut_enr_go)
mut_enr <- rbind.data.frame(mut_enr_ch,mut_enr_go)
dim(mut_enr)
write.csv(mut_enr,file="mut_enr_metagenes.csv")
mp = "Meta-program n"
cohort = "Sarcoma"
enrichdbs = "Cancer_Hallmarks_and_Gene_Ontology"
topn = "Top100"
h_mut_enr1 <- mut_enr[1:25,]#[1:500,]
h_mut_enr <- h_mut_enr1


Enrichr ... 
Connection is Live!

FlyEnrichr ... 
Connection is available!

WormEnrichr ... 
Connection is available!

YeastEnrichr ... 
Connection is available!

FishEnrichr ... 
Connection is available!

OxEnrichr ... 
Connection is available!

Connection changed to https://maayanlab.cloud/Enrichr/

Connection is Live!



,geneCoverage,genesPerTerm,libraryName,link,numTerms,appyter,categoryId
,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,13362,275,Genome_Browser_PWMs,http://hgdownload.cse.ucsc.edu/goldenPath/hg18/database/,615,ea115789fcbf12797fd692cec6df0ab4dbc79c6a,1
2,27884,1284,TRANSFAC_and_JASPAR_PWMs,http://jaspar.genereg.net/html/DOWNLOAD/,326,7d42eb43a64a4e3b20d721fc7148f685b53b6b30,1
3,6002,77,Transcription_Factor_PPIs,,290,849f222220618e2599d925b6b51868cf1dab3763,1
4,47172,1370,ChEA_2013,http://amp.pharm.mssm.edu/lib/cheadownload.jsp,353,7ebe772afb55b63b41b79dd8d06ea0fdd9fa2630,7
5,47107,509,Drug_Perturbations_from_GEO_2014,http://www.ncbi.nlm.nih.gov/geo/,701,ad270a6876534b7cb063e004289dcd4d3164f342,7
6,21493,3713,ENCODE_TF_ChIP-seq_2014,http://genome.ucsc.edu/ENCODE/downloads.html,498,497787ebc418d308045efb63b8586f10c526af51,7


Warning message in file(file, "rt"):
“cannot open file 't100_genes_per_metaprogram.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection
